# Opentron2 Proxy Viscometer Code w/ Custom Functions 

# Req. Packages/Modules

In [9]:
from opentrons import protocol_api
from opentrons import protocols
from opentrons import types

import numpy as np
import json 

import building_blocks as bb
import calculate_uncertainties as calcunc
import meniscus_tracking as mt
import custom_pipetting as cusp

import opentrons.execute
import opentrons.simulate


## System Start (acc documentation)

In [2]:
#systemctl start opentrons-robot-server

## Run Protocol Function (Dummy run, 3 Trials per sample, repeat)

In [3]:
def run(protocol: protocol_api.ProtocolContext):
    
    #Define tuberack
    CALAB_8_TUBERACK_20000UL_DEF_JSON = """{"ordering":[["A1","B1"],["A2","B2"],["A3","B3"],["A4","B4"]],
    "brand":{"brand":"CALAB","brandId":[]},
    "metadata":{"displayName":"CALAB 8 Tube Rack with Generic 20 mL","displayCategory":"tubeRack","displayVolumeUnits":"µL","tags":[]},
    "dimensions":{"xDimension":127.76,"yDimension":85.47,"zDimension":60.75},
    "wells":{"A1":{"depth":55.55,"totalLiquidVolume":20000,"shape":"circular","diameter":27.15,"x":16.5,"y":68.97,"z":5.2},
    "B1":{"depth":55.55,"totalLiquidVolume":20000,"shape":"circular","diameter":27.15,"x":16.5,"y":37.97,"z":5.2},
    "A2":{"depth":55.55,"totalLiquidVolume":20000,"shape":"circular","diameter":27.15,"x":47.5,"y":68.97,"z":5.2},
    "B2":{"depth":55.55,"totalLiquidVolume":20000,"shape":"circular","diameter":27.15,"x":47.5,"y":37.97,"z":5.2},
    "A3":{"depth":55.55,"totalLiquidVolume":20000,"shape":"circular","diameter":27.15,"x":78.5,"y":68.97,"z":5.2},
    "B3":{"depth":55.55,"totalLiquidVolume":20000,"shape":"circular","diameter":27.15,"x":78.5,"y":37.97,"z":5.2},
    "A4":{"depth":55.55,"totalLiquidVolume":20000,"shape":"circular","diameter":27.15,"x":109.5,"y":68.97,"z":5.2},
    "B4":{"depth":55.55,"totalLiquidVolume":20000,"shape":"circular","diameter":27.15,"x":109.5,"y":37.97,"z":5.2}},
    "groups":[{"brand":{"brand":"Generic","brandId":[]},"metadata":{"wellBottomShape":"flat","displayCategory":"tubeRack"},
    "wells":["A1","B1","A2","B2","A3","B3","A4","B4"]}],
    "parameters":{"format":"irregular","quirks":[],"isTiprack":false,"isMagneticModuleCompatible":false,"loadName":"calab_8_tuberack_20000ul"},
    "namespace":"custom_beta","version":1,"schemaVersion":2,"cornerOffsetFromSlot":{"x":0,"y":0,"z":0}}"""

    calab_8_tuberack_20000ul = json.loads(CALAB_8_TUBERACK_20000UL_DEF_JSON)

    # Define tips
    #tiprack_1 = protocol.load_labware('opentrons_96_tiprack_300ul', 11, label='tiprack')
    #tiprack_1.set_offset(x=0.8, y=-0.7, z=0.6)

    tiprack_2 = protocol.load_labware('opentrons_96_tiprack_1000ul',11, label='tiprack')
    tiprack_2.set_offset(x=0.0, y=2.0, z=2.5)
    # Define reservoir 
    reservoir = protocol.load_labware_from_definition(calab_8_tuberack_20000ul, 10,label='reservoir') 
    reservoir.set_offset(x=-2.0, y=2.0, z=0)

    # Define plate
    plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 5,label='plate')      
    plate.set_offset(x=-1.0, y=1.1, z=1.4)

    # Define pipettes
    #p300 = protocol.load_instrument('p300_single_gen2', mount='left', tip_racks=[tiprack_1])
    p1000 = protocol.load_instrument ('p1000_single_gen2', mount='right', tip_racks=[tiprack_2])
    bb.set_pipette_uncertainties(p1000, calcunc.call_p1000_error_to_vu())
    

    #************ Initialization Parameters ************
    pipette = p1000
    samples_ran = 0
    no_of_samples = 1 

    adepth = 0 
    depth = -50 #not used anymore 

    asptime = 5
    disptime = 5
    
    fr_arr = [10] # dispensed flowrate in uL/s #not used 
    well_touch_depth = 0
    touch_tip_speed = 30 

    res_letter_list = ["A", "B"]  #reservoir positions (of the custom defined calab_8_tuberack_20000ul)
    well_letter_list = ["A", "B", "C", "D", "E", "F", "G", "H"] #well positions (of the corning_96_wellplate_16.8ml_flat) --> 8 x 12 well plate 
    tube_rack_list = [f'{row}{col}' for row in 'AB' for col in range(1,5)]
    well_plate_list = [f"{row}" for row in "ABCDEFGH"]
    well_ind_list= [f"{col}" for col in range (1, 13)]
    #Samples will be in tubes in A1, A2, ..., A4, B1, ..., B4 of the tuberack "calab_8_tuberack_2000ul"
    #They will be dispensed in wells A1, ..., A12, H1, ..., H12 of the plate "plate"
    
    #************  ************
    # initiate the wells for each well
    for well in reservoir.wells():
        bb.initiate_well(well, mt.call_gradations_vial_20ml())
    for well in plate.wells():
        bb.initiate_well(well, mt.call_gradations_corning_360ul())

    shampoo = reservoir['A1']
    body_wash = reservoir['A2']
    diluted_body_wash = reservoir['A3']

    #i have no idea if this is necessary, added it because i kept getting an error about uninitialized volume even tho this wasn't in the example code on their github
    bb.set_headroom(shampoo, 10)
    bb.set_headroom(body_wash, 10)
    bb.set_headroom(diluted_body_wash, 10)
    bb.set_volume(shampoo, 20000)
    bb.set_volume(body_wash, 20000)
    bb.set_volume(diluted_body_wash, 20000)

    protocol.set_rail_lights(on=True)
    if p1000.has_tip:
        p1000.drop_tip()
    i = 0
#     #dummy run - systematic error correction of exterior excess weight
#     if i == 0:
#         pipette.pick_up_tip() #pick up pipette tip from tip rack 

#         #move to reservoir 
#         pipette.move_to(reservoir[tube_rack_list[samples_ran]].top()) #moves the pipette tip to the top of the reservoir tube  (depth of 55.55 for the CALAB) 
#         protocol.delay(seconds=2) #actual trial should delay for 10

#         cusp.custom_touch_tip(pipette, well=reservoir[tube_rack_list[samples_ran]], radius_offset=1, depth=0, speed=touch_tip_speed, increments=2)

#         #move to plate 
#         pipette.move_to(plate[well_plate_list[samples_ran] + well_ind_list[0]].top())    # move to top of well plate
#         protocol.delay(seconds=2) #actual trial should delay for 5

#         #custom_touch_tip(pipette=p300, well=reservoir[tube_rack_list[samples_ran]], radius_offset=1, depth=-10, speed=touch_tip_speed, increments=2) 
#         pipette.move_to(reservoir[tube_rack_list[samples_ran]].top())
#         #protocol.pause("Time to measure.")  # take mass reading (will be the empty plate only during trial run)

#         cusp.custom_touch_tip(pipette, well=reservoir[tube_rack_list[samples_ran]], radius_offset=1, depth=-10, speed=touch_tip_speed, increments=2)
#         for e in range(10):
#             pipette.blow_out() 
#         protocol.delay(5)
#         for s in range(10):
#             pipette.blow_out()
        
#         pipette.drop_tip()

#         #for actual runs, 3 trials each measurement

    print(f'0: Initialized all the variables  ')

    while samples_ran < no_of_samples:
        for t in range(3):
            print(f'1: Picking up the pipette tip')
            pipette.pick_up_tip()
            trans_vol = 1000
            imm_depth = 5
            safe_height = 0
            flowrate = 273.15
            
            res_loc = reservoir[tube_rack_list[samples_ran]]
            print(f'2: Aspirating now from {res_loc} at a rate of {flowrate}')
            cusp.custom_aspirate(pipette, transfer_volume=trans_vol, location=res_loc, immersion_depth=imm_depth, safety_height=safe_height, rate=flowrate) #avoids submerging the tip in the liquid
            
            #add a move up before waiting 
            print(f'3: Moving up, before delaying')
            pipette.move_to(res_loc.top())
                
            
            #wait for the excess to drip off
            print(f'4: in a pause to drip off excess')
            if trans_vol <= 100:
                protocol.delay(seconds=10) 
            elif 100 < trans_vol <= 600:  
                protocol.delay(seconds=20)          
            else:
                protocol.delay(seconds=30)

            #wipe off excess on edge
            
            print(f' 5: touching tip on edge of {res_loc} twice')
            cusp.custom_touch_tip(pipette, well=res_loc, radius_offset=1, depth=-10, speed=touch_tip_speed, increments=2)

            #dispense
            plate_loc = plate[(well_plate_list[samples_ran] + well_ind_list[t])]
            print(f'6: im gonna dispense at {plate_loc}')
            cusp.custom_dispense(pipette, transfer_volume=trans_vol, location=plate_loc, immersion_depth=imm_depth, rate=flowrate)

            #wait for the excess to drip off
            print(f'7: waiting for excess to drip off')
            protocol.delay(seconds=10)
            print(f'8: i am moving now to the {res_loc})
            pipette.move_to(res_loc.top()) #moves the pipette tip to the top of the reservoir tube  

            
            #protocol.pause() #pause to measure the liquid dipsened on the well plate; resume once done measuring the change in mass and place the plate back
            print(f'9: i am blowing out now')
            for e in range(10):
                pipette.blow_out() 
            protocol.delay(5)
            for s in range(10):
                pipette.blow_out()

            print(f'10: i am dropping the tip in the trash now')
            pipette.drop_tip() #drop the pipette tip into trash

        print(f'11: increment samples by 1')
        samples_ran += 1

## Create Protocol Instance and Call Run Function

In [ ]:
protocol = opentrons.execute.get_protocol_api("2.10")
protocol.home()

run(protocol)

import opentrons.simulate
protocol = opentrons.simulate.get_protocol_api("2.10")
run(protocol)
opentrons.simulate.simulate(protocol)